<a href="https://colab.research.google.com/github/bbaaxx/translated-opt175b-gen/blob/add-notebook/OPT_175B_%2B_Translation_%5Bcpu%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#OPT-175B + Traducción bidireccional

Este notebook contiene una implementación simple de el modelo [OPT-175B](https://arxiv.org/abs/2205.01068) con traducción bi-direcional utilizando 2  [transformadores  universidad de Helsinki](https://huggingface.co/Helsinki-NLP) para traducir hacia y desde el lenguaje nativo del modelo OPT.

Este notebook es utilizado para presentar un ejemplo comentado en español, de una implementación sencillo dpara obtener generacion de texto en diferentes idiomas.

##Instalar y configurar cosas necesarias
----------------------------------------
(OJO: Ajustar configuraciones globales aquí)

In [ ]:
# Lenguajes de entrada y salida (pueden ser distintos meintras que != wn )
INPUT_LANG = "es"
OUTPUT_LANG = "es"

# Del mas chico al mas grande, el tiempo de procesamiento es proporcional al
# tamaño del modelo que decidamos usar, algunos ejemplos:
# "facebook/opt-125m" | "facebook/opt-350m" | "facebook/opt-1.3b | "facebook/opt-2.7b" 
GEN_MODEL = "facebook/opt-350m"

# Cuantas palabras maximo pueden generarse
GEN_TOKENS = 40

# Resultados que seran generados
GEN_RESULTS = 5


###Dependencias



No es necesario usar una GPU asi que podemos instalar directamente:

1.   transformers
2.   torch (instalado en el entorno)
3.   sentencepiece
4.   sacremoses


In [ ]:
!pip3 install transformers sentencepiece sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


###Pipelines de traducción

Vamos a usar los impresionantes [modelos del LTRG de la universidad de Helsinki](https://huggingface.co/Helsinki-NLP) para traducir hacia y desde el lenguaje nativo del modelo OPT (Inglés)

In [ ]:
from transformers import pipeline

def getTranslator(src, dst):
  task_name = f"translation_{src}_to_{dst}"
  model_name = f"Helsinki-NLP/opus-mt-{src}-{dst}"
  return pipeline(task_name, model=model_name, tokenizer=model_name)

translate_to_en = getTranslator(INPUT_LANG, "en")
translate_from_en = getTranslator("en", OUTPUT_LANG)

Downloading:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/298M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/807k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/783k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/298M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/783k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/807k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52M [00:00<?, ?B/s]

### Pipeline de generación (OPT)


In [ ]:
from transformers import set_seed

set_seed(16)
gen_pipeline = False # This is here to free up some emory before rebuilding the generator
gen_pipeline = pipeline('text-generation', model=GEN_MODEL, do_sample=True, max_length=GEN_TOKENS, num_return_sequences=GEN_RESULTS)

Downloading:   0%|          | 0.00/613 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/632M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/721 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/441 [00:00<?, ?B/s]

#Agregamos algo de azucar

De la que no hace daño: azucar sintáctica

In [ ]:
import pprint

pp = pprint.PrettyPrinter(indent=2).pprint

def processPrompt(prompt):
  gen = [translate_from_en(x['generated_text']) for x in gen_pipeline(translate_to_en(prompt)[0]['translation_text'])]
  return [x[0]['translation_text'] for x in gen]

#Usemos el sistema

In [ ]:
input = "La forma mas efectiva de combatir el hambre en el mundo "
output = processPrompt(input)
for x in output:
  pp(x)

('La manera más efectiva de luchar contra el hambre en el mundo es dejar de '
 'comer granos. Sé que no hay un gran número de ellos, o cualquier otro '
 'recurso, pero aún así.')
('La manera más eficaz de luchar contra el hambre en el mundo es cultivar su '
 'propia comida (véase también: la Revolución Alimentaria del USDA). Y porque '
 'nunca he visto una comida tan deliciosa y satisfactoria,')
('La manera más eficaz de luchar contra el hambre en el mundo (junto con la '
 'cocina) es comer comidas pequeñas y tomar descansos. La idea de que el '
 'hambre es normal y manejable tiene sentido y ayuda a explicar')
('La manera más efectiva de luchar contra el hambre en el mundo es a través de '
 'la comida! No u Bueno estoy en Australia y conseguimos medio paquete de '
 'nueces al día. También estoy en la dieta')
('La manera más eficaz de luchar contra el hambre en el mundo de hoy es hacer '
 'que los alimentos sean seguros para nuestra gente. Necesitamos apoyar a '
 'nuestros profesionales

In [ ]:
input = "Una excelente forma de erradicar el hambre en el mundo sería "
output = processPrompt(input)
for x in output:
  pp(x)

('Una excelente manera de erradicar el hambre en el mundo sería que todo el '
 'mundo tuviera un solo alimento con un conteo de calorías para ayudarle a '
 'mantenerse delgado. Pero entonces, ¿quién pagaría por cada artículo?')
('Una excelente manera de erradicar el hambre en el mundo sería fijar el '
 'salario mínimo nacional en 50 centavos por hora para los trabajadores con '
 'salarios bajos.')
('Una excelente manera de erradicar el hambre en el mundo sería reducir este '
 'desperdicio de recursos. Ese sería mi plan, pero mi escuela tiene el '
 'requisito de que pueda hacer la comida más barata, así que')
('Una excelente manera de erradicar el hambre en el mundo sería elevar todos '
 'los precios de los alimentos a cero, pero a medida que la crisis alimentaria '
 'empeora, también lo hace el precio de comer.')
('Una excelente manera de erradicar el hambre en el mundo sería mejorando el '
 'transporte y la educación pública en las escuelas. Esto no sólo evitaría el '
 'hambre en los 

In [ ]:
input = "Para terminar con la pobreza en el mundo lo mejor es "
output = processPrompt(input)
for x in output:
  pp(x + '\n')

('Para acabar con la pobreza en el mundo lo mejor es abolir la esclavitud y '
 'acabar con la pobreza en todos los países del mundo.\n')
('Para acabar con la pobreza en el mundo lo mejor es no tener que comer y '
 'sobrevivir! Hoy es el primer día del año escolar en la India.\n')
('Para acabar con la pobreza en el mundo lo mejor es conseguir comida sobre la '
 'mesa para todos, no sólo para los ricos. Entonces, ¿qué hay de eso? Lo mejor '
 'es acabar con la pobreza en\n')
('Para acabar con la pobreza en el mundo lo mejor es dejar de matar a la '
 'gente.\n')
('Para acabar con la pobreza en el mundo lo mejor es implementar un ingreso '
 'básico universal. ¿Por qué no un ingreso básico universal? ¿Qué pasa con la '
 'atención sanitaria universal? ¿Son mejores? Creo que queremos\n')


In [ ]:
input = "Una excelente forma de erradicar la pobreza es: "
output = processPrompt(input)
for x in output:
  pp(x + '\n')

('Una excelente manera de erradicar la pobreza es: ¡pagar! Hacer frente a la '
 'pobreza no es fácil. Cuando estás en medio de la nada, no es difícil de '
 'hacer.\n')
('Una excelente manera de erradicar la pobreza es: “los pueblos del planeta '
 'deben trabajar duro para eliminar la pobreza y el hambre, en la vida de sus '
 'hijos y en la de otros pueblos.\n')
('Una excelente manera de erradicar la pobreza es: si una persona puede '
 'permitirse comprar una bolsa, comprar 4 bolsas más.\n')
('Una excelente manera de erradicar la pobreza es: * Haz que la gente done '
 'dinero * Muéstrales que incluso un poco podría tener un impacto en su mundo '
 'y ayudarles a vivir con más dignidad >Get\n')
('Una excelente manera de erradicar la pobreza es: cortar los alimentos, '
 'cupones de alimentos, subsidios de vivienda, etc. El gobierno federal ya lo '
 'hace y sólo tiene que empezar a gastar en otras cosas.\n')


In [ ]:
input = "el fin justifica los medios siempre y cuando "
output = processPrompt(input)
for x in output:
  pp(x + '\n')

('el fin justifica los medios siempre y cuando el proceso apoya los fines, y '
 'si es así, entonces bueno para usted. No estoy en desacuerdo. ¿Qué es una '
 'solución a todo esto? Este sistema\n')
('el fin justifica los medios mientras que eso es lo que hacen las personas '
 'que lo quieren.\n')
('el fin justifica los medios mientras el perpetrador sea ahorcado Esto se '
 'llama "El Daño Duradero".\n')
('el fin justifica los medios siempre y cuando se haga correctamente. No el '
 'objetivo es poner fin a la explotación de los seres humanos - o alguna otra '
 'especie, si ese es un término apropiado para lo que los creadores\n')
('el fin justifica los medios siempre y cuando no sea en contra de su '
 'voluntad. Me gustaría que la gente se diera cuenta de esto. En América, '
 'usted tendría que tener el derecho de votar en contra de la voluntad de la '
 'gente.\n')


In [ ]:
input = "La friendzone es un mito porque"
output = processPrompt(input)
for x in output:
  pp(x + '\n')

('Friendzone es un mito porque por lo general ni siquiera lo reconocen. Los '
 '"amigos" son del otro día, por lo que no es completamente aleatorio.\n')
('Friendzone es un mito porque fue hecho por hombres heterosexuales, que '
 'siempre estuvieron dispuestos a ser groseros cuando eran una mujer, siempre '
 'y cuando tuvieron el valor de admitirlo.\n')
('Friendzone es un mito debido a la idea de que en su mente una relación debe '
 'ser un libre-para-todos de cualquier y todas las emociones / sentimientos.\n')
('Friendzone es un mito porque en este submarino mucha gente tiene cero '
 'interés en ver/hablar con las mujeres. Apuesto a que tienes la misma '
 'mentalidad cuando se trata de chicos. Empezaría\n')
('Friendzone es un mito porque no hay sitios de citas que permiten cualquier '
 'tipo de citas en línea donde usted tiene una "cita" con las niñas. He '
 'encontrado más que algunas chicas que usted\n')


In [ ]:
input = "Esto es lo que debo hacer para salir de la friendzone"
output = processPrompt(input)
for x in output:
  pp(x + '\n')

('Esto es lo que tengo que hacer para salir de la zona de amigos, la gente '
 'necesita tener cuidado, es todo lo que pueden hacer para superar este '
 'desastre.\n')
('Esto es lo que tengo que hacer para salir de la zona de amigos. Creo que he '
 'desarrollado este único hábito toda mi vida.\n')
'Esto es lo que tengo que hacer para salir de la zona de amigos.\n'
('Esto es lo que tengo que hacer para salir de la zona de amigos. Quiero ser '
 'capaz de existir de mi propia manera única que realmente no quiero ser. '
 'Quiero conocer\n')
'Esto es lo que tengo que hacer para salir de la zona de amigos.\n'


In [ ]:
input = "estas son cosas que te pasan cuando estas en la friendzone, por ejemplo: "
output = processPrompt(input)
for x in output:
  pp(x)

('Estas son las cosas que te suceden cuando estás en la zona de amigos, por '
 'ejemplo: Si te gusta una chica que está en la zona de amigos, es probable '
 'que encuentres que puede no tener')
('Estas son cosas que te pasan cuando estás en la zona de amigos, por ejemplo: '
 '* una chica, pero no * un chico * un chico sin ninguna razón para ti * un '
 'chico (')
('Estas son las cosas que te pasan cuando estás en la zona de amigos, por '
 'ejemplo: - ser una maldita virgen o algo así. >ser una maldita virgen ¿Es '
 'eso una cosa?')
('Estas son las cosas que te pasan cuando estás en la zona de amigos, por '
 'ejemplo: 1. Creo que podría ser atraído por ti 2. Haríamos mierda que me '
 'hace no')
('Estas son cosas que te pasan cuando estás en la zona de amigos, por ejemplo: '
 '"¿Y si entraste en la casa de la chica y yo conducía un taxi en su coche?" '
 '"¿Qué?')
